In [1]:
import pandas as pd
import nltk
import numpy as np
import re
from tqdm.notebook import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.utils import plot_model
import tensorflow as tf
import random
from gensim.models import Word2Vec
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


In [2]:
# Load the dataset
data = pd.read_csv('train_balanced.csv')

# Filter relevant columns
data = data[['label', 'comment']]

In [3]:
from tokenizers import ByteLevelBPETokenizer
import nltk
from nltk.stem import WordNetLemmatizer

# Assuming data is a DataFrame with a column named "comment"
data["comment"] = data["comment"].astype(str)
texts = data["comment"].tolist()

# Initialize the BPE tokenizer
tokenizer = ByteLevelBPETokenizer()

# Train the tokenizer on the text data (this step learns the BPE vocabulary)
tokenizer.train_from_iterator(texts)

# Encode the entire DataFrame column 'comment' into subword tokens
encoded_texts = [tokenizer.encode(text).ids for text in texts]

# Decode the subword tokens back into text for the first element in the DataFrame column
decoded_text = tokenizer.decode(encoded_texts[0])




In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tahor\AppData\Roaming\nltk_data...


True

In [14]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Tahor\AppData\Roaming\nltk_data...


True

In [15]:
from tokenizers import ByteLevelBPETokenizer
import nltk
from nltk.stem import WordNetLemmatizer


lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Lemmatize each word in the text
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the lemmatized words back to form the lemmatized text
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

# Lemmatize the decoded text
lemmatized_text = lemmatize_text(decoded_text)
print(lemmatized_text)

NC and NH .


In [16]:
import nltk
from nltk.corpus import stopwords
from tokenizers import ByteLevelBPETokenizer
from nltk.stem import WordNetLemmatizer

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tahor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop_words = set(stopwords.words('english'))  # Assuming you are working with English text

# Tokenize the lemmatized text into words
words = nltk.word_tokenize(lemmatized_text)

# Remove stop words from the tokenized words
filtered_words = [word for word in words if word.lower() not in stop_words]

# Join the filtered words back to form the final text without stop words
final_text = ' '.join(filtered_words)
print(final_text)

NC NH .


In [4]:
# Function for text cleaning and correction
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
   
     # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text



In [5]:
# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)